In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 4
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC50(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC50(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC50(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.26 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        
        #------------------------------------------------
        #需要調整的地方
        #------------------------------------------------
        #--要用幾秒訓練 data_1S/data_3S/data_1S
        #--學習率
        #--epoch
        if(TL==0):
            trainData = data_5S
            epoch = 120
            lr = 0.00032
        elif(TL==1):
            trainData = data_3S
            epoch = 120
            lr = 0.00032
        else:
            trainData = data_1S
            epoch = 120
            lr = 0.00032
        #------------------------------------------------
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型
        model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
        #測試準確度
        test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
        test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
        test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/120 Train_Loss: 2.6856114864349365 Train_acc: 18.500000%
Train Epoch: 2/120 Train_Loss: 1.9816739559173584 Train_acc: 44.875000%
Train Epoch: 3/120 Train_Loss: 1.206974744796753 Train_acc: 63.312500%
Train Epoch: 4/120 Train_Loss: 0.9292609691619873 Train_acc: 78.062500%
Train Epoch: 5/120 Train_Loss: 0.5748963952064514 Train_acc: 87.375000%
Train Epoch: 6/120 Train_Loss: 0.33067286014556885 Train_acc: 94.812500%
Train Epoch: 7/120 Train_Loss: 0.13756315410137177 Train_acc: 97.812500%
Train Epoch: 8/120 Train_Loss: 0.1488763839006424 Train_acc: 98.625000%
Train Epoch: 9/120 Train_Loss: 0.047189947217702866 Train_acc: 98.937500%
Train Epoch: 10/120 Train_Loss: 0.10683564096689224 Train_acc: 98.937500%
Train Epoch: 11/120

Train Epoch: 106/120 Train_Loss: 0.001303785596974194 Train_acc: 100.000000%
Train Epoch: 107/120 Train_Loss: 0.001130241435021162 Train_acc: 100.000000%
Train Epoch: 108/120 Train_Loss: 0.0017459859373047948 Train_acc: 100.000000%
Train Epoch: 109/120 Train_Loss: 0.0017047255532816052 Train_acc: 100.000000%
Train Epoch: 110/120 Train_Loss: 0.0010365392081439495 Train_acc: 100.000000%
Train Epoch: 111/120 Train_Loss: 0.0013710296479985118 Train_acc: 100.000000%
Train Epoch: 112/120 Train_Loss: 0.0011476275976747274 Train_acc: 100.000000%
Train Epoch: 113/120 Train_Loss: 0.001612428342923522 Train_acc: 100.000000%
Train Epoch: 114/120 Train_Loss: 0.0019656179938465357 Train_acc: 100.000000%
Train Epoch: 115/120 Train_Loss: 0.037734922021627426 Train_acc: 99.625000%
Train Epoch: 116/120 Train_Loss: 0.024706460535526276 Train_acc: 99.312500%
Train Epoch: 117/120 Train_Loss: 0.04021840915083885 Train_acc: 99.250000%
Train Epoch: 118/120 Train_Loss: 0.013199632056057453 Train_acc: 99.875000

Train Epoch: 92/120 Train_Loss: 0.13994300365447998 Train_acc: 97.416672%
Train Epoch: 93/120 Train_Loss: 0.11585289239883423 Train_acc: 97.416672%
Train Epoch: 94/120 Train_Loss: 0.08175043761730194 Train_acc: 97.458336%
Train Epoch: 95/120 Train_Loss: 0.15197236835956573 Train_acc: 97.437500%
Train Epoch: 96/120 Train_Loss: 0.06003232300281525 Train_acc: 97.437500%
Train Epoch: 97/120 Train_Loss: 0.07853052765130997 Train_acc: 97.458336%
Train Epoch: 98/120 Train_Loss: 0.13229632377624512 Train_acc: 97.458336%
Train Epoch: 99/120 Train_Loss: 0.10460065305233002 Train_acc: 97.437500%
Train Epoch: 100/120 Train_Loss: 0.0069931806065142155 Train_acc: 97.458336%
Train Epoch: 101/120 Train_Loss: 0.11650612205266953 Train_acc: 97.458336%
Train Epoch: 102/120 Train_Loss: 0.1403440237045288 Train_acc: 97.458336%
Train Epoch: 103/120 Train_Loss: 0.03015706315636635 Train_acc: 97.416672%
Train Epoch: 104/120 Train_Loss: 0.1470603048801422 Train_acc: 97.479172%
Train Epoch: 105/120 Train_Loss: 

Train Epoch: 78/120 Train_Loss: 0.2206515669822693 Train_acc: 93.262505%
Train Epoch: 79/120 Train_Loss: 0.29555588960647583 Train_acc: 93.262505%
Train Epoch: 80/120 Train_Loss: 0.2020064890384674 Train_acc: 93.262505%
Train Epoch: 81/120 Train_Loss: 0.3686677813529968 Train_acc: 93.275002%
Train Epoch: 82/120 Train_Loss: 0.22142978012561798 Train_acc: 92.812508%
Train Epoch: 83/120 Train_Loss: 0.3561941683292389 Train_acc: 91.550003%
Train Epoch: 84/120 Train_Loss: 0.04602435976266861 Train_acc: 93.112503%
Train Epoch: 85/120 Train_Loss: 0.21499839425086975 Train_acc: 93.225006%
Train Epoch: 86/120 Train_Loss: 0.3062247633934021 Train_acc: 93.237503%
Train Epoch: 87/120 Train_Loss: 0.20268702507019043 Train_acc: 93.262505%
Train Epoch: 88/120 Train_Loss: 0.17983849346637726 Train_acc: 93.275002%
Train Epoch: 89/120 Train_Loss: 0.4427223205566406 Train_acc: 93.275002%
Train Epoch: 90/120 Train_Loss: 0.33671310544013977 Train_acc: 93.262505%
Train Epoch: 91/120 Train_Loss: 0.2296844273

Train Epoch: 63/120 Train_Loss: 0.0014381472719833255 Train_acc: 100.000000%
Train Epoch: 64/120 Train_Loss: 0.0013139144284650683 Train_acc: 100.000000%
Train Epoch: 65/120 Train_Loss: 0.0022974626626819372 Train_acc: 100.000000%
Train Epoch: 66/120 Train_Loss: 0.0014614870306104422 Train_acc: 100.000000%
Train Epoch: 67/120 Train_Loss: 0.0019321510335430503 Train_acc: 100.000000%
Train Epoch: 68/120 Train_Loss: 0.0010303943417966366 Train_acc: 100.000000%
Train Epoch: 69/120 Train_Loss: 0.0014068176969885826 Train_acc: 100.000000%
Train Epoch: 70/120 Train_Loss: 0.0012365540023893118 Train_acc: 100.000000%
Train Epoch: 71/120 Train_Loss: 0.0014244243502616882 Train_acc: 100.000000%
Train Epoch: 72/120 Train_Loss: 0.0029373562429100275 Train_acc: 100.000000%
Train Epoch: 73/120 Train_Loss: 0.002253506099805236 Train_acc: 100.000000%
Train Epoch: 74/120 Train_Loss: 0.0014643548056483269 Train_acc: 100.000000%
Train Epoch: 75/120 Train_Loss: 0.002061977284029126 Train_acc: 100.000000%
T

...

Train Epoch: 54/120 Train_Loss: 0.40426868200302124 Train_acc: 92.750008%
Train Epoch: 55/120 Train_Loss: 0.4604128301143646 Train_acc: 92.787506%
Train Epoch: 56/120 Train_Loss: 0.27660778164863586 Train_acc: 92.787506%
Train Epoch: 57/120 Train_Loss: 0.2103695273399353 Train_acc: 92.712502%
Train Epoch: 58/120 Train_Loss: 0.13026686012744904 Train_acc: 92.737503%
Train Epoch: 59/120 Train_Loss: 0.27332428097724915 Train_acc: 92.725006%
Train Epoch: 60/120 Train_Loss: 0.30697494745254517 Train_acc: 92.750008%
Train Epoch: 61/120 Train_Loss: 0.1207895427942276 Train_acc: 92.737503%
Train Epoch: 62/120 Train_Loss: 0.2688221037387848 Train_acc: 92.787506%
Train Epoch: 63/120 Train_Loss: 0.26309752464294434 Train_acc: 92.775002%
Train Epoch: 64/120 Train_Loss: 0.12285175919532776 Train_acc: 92.700005%
Train Epoch: 65/120 Train_Loss: 0.18248917162418365 Train_acc: 92.787506%
Train Epoch: 66/120 Train_Loss: 0.3465839922428131 Train_acc: 92.725006%
Train Epoch: 67/120 Train_Loss: 0.284399479

Train Epoch: 40/120 Train_Loss: 0.006949419155716896 Train_acc: 100.000000%
Train Epoch: 41/120 Train_Loss: 0.00494487164542079 Train_acc: 100.000000%
Train Epoch: 42/120 Train_Loss: 0.010545391589403152 Train_acc: 100.000000%
Train Epoch: 43/120 Train_Loss: 0.005683952011168003 Train_acc: 100.000000%
Train Epoch: 44/120 Train_Loss: 0.0038858402986079454 Train_acc: 100.000000%
Train Epoch: 45/120 Train_Loss: 0.0041975886560976505 Train_acc: 100.000000%
Train Epoch: 46/120 Train_Loss: 0.0036155616398900747 Train_acc: 100.000000%
Train Epoch: 47/120 Train_Loss: 0.0025779344141483307 Train_acc: 100.000000%
Train Epoch: 48/120 Train_Loss: 0.004105862695723772 Train_acc: 100.000000%
Train Epoch: 49/120 Train_Loss: 0.004047279711812735 Train_acc: 100.000000%
Train Epoch: 50/120 Train_Loss: 0.003919617738574743 Train_acc: 100.000000%
Train Epoch: 51/120 Train_Loss: 0.002838392509147525 Train_acc: 100.000000%
Train Epoch: 52/120 Train_Loss: 0.004394811112433672 Train_acc: 100.000000%
Train Epo

Train Epoch: 23/120 Train_Loss: 0.12797954678535461 Train_acc: 95.208336%
Train Epoch: 24/120 Train_Loss: 0.15328338742256165 Train_acc: 97.000000%
Train Epoch: 25/120 Train_Loss: 0.12168318033218384 Train_acc: 97.000000%
Train Epoch: 26/120 Train_Loss: 0.06699328869581223 Train_acc: 97.229172%
Train Epoch: 27/120 Train_Loss: 0.12630699574947357 Train_acc: 97.000000%
Train Epoch: 28/120 Train_Loss: 0.28748229146003723 Train_acc: 95.916672%
Train Epoch: 29/120 Train_Loss: 0.21708178520202637 Train_acc: 95.708336%
Train Epoch: 30/120 Train_Loss: 0.3316531777381897 Train_acc: 95.354172%
Train Epoch: 31/120 Train_Loss: 0.17682825028896332 Train_acc: 96.833336%
Train Epoch: 32/120 Train_Loss: 0.08046218752861023 Train_acc: 97.312500%
Train Epoch: 33/120 Train_Loss: 0.0028803509194403887 Train_acc: 97.291672%
Train Epoch: 34/120 Train_Loss: 0.0816132128238678 Train_acc: 97.270836%
Train Epoch: 35/120 Train_Loss: 0.055558204650878906 Train_acc: 97.250000%
Train Epoch: 36/120 Train_Loss: 0.064

Train Epoch: 9/120 Train_Loss: 0.5151402354240417 Train_acc: 90.012505%
Train Epoch: 10/120 Train_Loss: 0.3278096616268158 Train_acc: 91.100006%
Train Epoch: 11/120 Train_Loss: 0.46614909172058105 Train_acc: 91.187508%
Train Epoch: 12/120 Train_Loss: 0.2647954523563385 Train_acc: 90.775002%
Train Epoch: 13/120 Train_Loss: 0.3941417634487152 Train_acc: 90.650002%
Train Epoch: 14/120 Train_Loss: 0.29374226927757263 Train_acc: 90.600006%
Train Epoch: 15/120 Train_Loss: 0.27162933349609375 Train_acc: 90.912506%
Train Epoch: 16/120 Train_Loss: 0.2980195879936218 Train_acc: 91.475006%
Train Epoch: 17/120 Train_Loss: 0.39369598031044006 Train_acc: 90.100006%
Train Epoch: 18/120 Train_Loss: 0.2114218920469284 Train_acc: 90.100006%
Train Epoch: 19/120 Train_Loss: 0.31010860204696655 Train_acc: 90.850006%
Train Epoch: 20/120 Train_Loss: 0.2918396592140198 Train_acc: 91.037506%
Train Epoch: 21/120 Train_Loss: 0.2112269103527069 Train_acc: 91.787506%
Train Epoch: 22/120 Train_Loss: 0.3010042011737

Test accuracy of the model: 47.350000%
Test accuracy of the model: 56.500000%
Test accuracy of the model: 62.000000%
Training Time: 393.07 seconds


In [5]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()